In [ ]:
import pydicom
import numpy as np
import torch
from torchvision import transforms

# X-ray 전처리 함수
def preprocess_xray(xray_path):
    """
    X-ray 이미지를 전처리하고 PyTorch 텐서로 변환하는 함수.

    Args:
        xray_path (str): X-ray DICOM 파일 경로.

    Returns:
        torch.Tensor: 전처리된 X-ray 이미지 텐서 [1, 1, height, width].
    """
    # 1. DICOM 파일 읽기
    dicom_data = pydicom.dcmread(xray_path)
    xray_image = dicom_data.pixel_array.astype(np.float32)

    # 2. 정규화 (값을 0~1 범위로 변환)
    xray_image -= xray_image.min()
    xray_image /= (xray_image.max() - xray_image.min())

    # 3. 차원 확장 [1, 1, height, width]
    xray_image = np.expand_dims(xray_image, axis=(0, 1))

    # 4. NumPy 배열을 PyTorch 텐서로 변환
    xray_tensor = torch.from_numpy(xray_image)

    # 5. 추가적인 전처리 (예: 리사이즈, 데이터 증강)
    # 리사이즈: 원하는 크기로 맞추기 (예: 320x320)
    resize_transform = transforms.Resize((320, 320))
    xray_tensor = resize_transform(xray_tensor)

    return xray_tensor


# 정면 X-ray
front_xray_path = '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-35511/3000923.000000-NA-62357/1-1.dcm'
front_xray_tensor = preprocess_xray(front_xray_path)

# 측면 X-ray
side_xray_path = '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-35511/3000923.000000-NA-62357/1-2.dcm'
side_xray_tensor = preprocess_xray(side_xray_path)

print(front_xray_tensor.shape)  # [1, 1, 320, 320]
print(side_xray_tensor.shape)  # [1, 1, 320, 320]

import os
import numpy as np
import pydicom
import gc
import torch
import SimpleITK as sitk
from scipy.ndimage import zoom

# DICOM 슬라이스 로드 함수
def load_ct_slices(ct_folder_path):
    slices = []
    slice_thickness = None
    pixel_spacing = None

    for filename in sorted(os.listdir(ct_folder_path)):
        if filename.endswith('.dcm'):
            filepath = os.path.join(ct_folder_path, filename)
            dicom_data = pydicom.dcmread(filepath)
            if slice_thickness is None:
                slice_thickness = float(dicom_data.SliceThickness)
            if pixel_spacing is None:
                pixel_spacing = [float(i) for i in dicom_data.PixelSpacing]
            slices.append(dicom_data.pixel_array)

    ct_volume = np.stack(slices, axis=0)  # [depth, height, width]
    return ct_volume, pixel_spacing, slice_thickness

# Resampling 함수
def resample(image, spacing, new_spacing=[1, 1, 1]):
    spacing = np.array(spacing)
    new_spacing = np.array(new_spacing)
    resize_factor = spacing / new_spacing
    new_shape = np.round(image.shape * resize_factor)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    image = zoom(image, real_resize_factor, mode='nearest')
    return image, new_spacing

# 표준 크기로 크롭하는 함수
def crop_to_standard(scan, scale=320):
    z, y, x = scan.shape
    if z >= scale:
        ret_scan = scan[z-scale:z, :, :]
    else:
        padding_top = (scale - z) // 2
        padding_bottom = scale - z - padding_top
        ret_scan = np.pad(scan, ((padding_top, padding_bottom), (0, 0), (0, 0)), mode='constant')
    z, y, x = ret_scan.shape
    if y >= scale:
        ret_scan = ret_scan[:, (y-scale)//2:(y+scale)//2, :]
    else:
        padding_left = (scale - y) // 2
        padding_right = scale - y - padding_left
        ret_scan = np.pad(ret_scan, ((0, 0), (padding_left, padding_right), (0, 0)), mode='constant')
    if x >= scale:
        ret_scan = ret_scan[:, :, (x-scale)//2:(x+scale)//2]
    else:
        padding_front = (scale - x) // 2
        padding_back = scale - x - padding_front
        ret_scan = np.pad(ret_scan, ((0, 0), (0, 0), (padding_front, padding_back)), mode='constant')
    return ret_scan

# CT 스캔 저장 함수
def save_scan_mhda(volume, origin, spacing, path):
    itkimage = sitk.GetImageFromArray(volume, isVector=False)
    itkimage.SetSpacing(spacing)
    itkimage.SetOrigin(origin)
    sitk.WriteImage(itkimage, path, True)

# CT 전처리 및 저장
def preprocess_ct(ct_folder_path, save_path, target_spacing=[1, 1, 1], crop_scale=320):
    ct_volume, pixel_spacing, slice_thickness = load_ct_slices(ct_folder_path)
    original_spacing = [slice_thickness] + pixel_spacing
    print("Original CT shape:", ct_volume.shape)
    print("Original spacing:", original_spacing)

    resampled_ct, new_spacing = resample(ct_volume, original_spacing, target_spacing)
    print("Resampled CT shape:", resampled_ct.shape)
    print("New spacing:", new_spacing)

    cropped_ct = crop_to_standard(resampled_ct, scale=crop_scale)
    print("Cropped CT shape:", cropped_ct.shape)

    save_scan_mhda(cropped_ct, (0, 0, 0), new_spacing, save_path)
    print("Processed CT saved at:", save_path)

    return cropped_ct, new_spacing

# 실행
if __name__ == "__main__":
    ct_folder_path = '/content/drive/MyDrive/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192'
    save_path = '/content/drive/MyDrive/processed_ct_volume.mha'

    # CT 전처리 및 크롭된 데이터 반환
    cropped_ct, new_spacing = preprocess_ct(ct_folder_path, save_path, target_spacing=[1, 1, 1], crop_scale=320)

    # PyTorch 텐서 변환
    cropped_ct = np.expand_dims(cropped_ct, axis=(0, 1))  # [1, 1, depth, height, width]
    ct_tensor = torch.from_numpy(cropped_ct).float()
    print("Final CT shape:", ct_tensor.shape)

    # CT 텐서 정규화 (-1 ~ 1 범위로 클리핑 후 0 ~ 1로 변환)
    print("Before normalization:")
    print(f"CT Volume Tensor Min: {ct_tensor.min()}, Max: {ct_tensor.max()}, Mean: {ct_tensor.mean()}")

    ct_tensor = torch.clamp(ct_tensor, min=-1000, max=1000)  # -1000 ~ 1000으로 클리핑
    ct_tensor = (ct_tensor + 1000) / 2000  # 0 ~ 1로 정규화

    print("After normalization:")
    print(f"CT Volume Tensor Min: {ct_tensor.min()}, Max: {ct_tensor.max()}, Mean: {ct_tensor.mean()}")

    # 메모리 정리
    del cropped_ct
    gc.collect()
    print("Memory cleared!")

import matplotlib.pyplot as plt
import numpy as np

def visualize_data(front_xray_tensor, side_xray_tensor, ct_volume_tensor):
    """
    전처리된 X-ray 및 CT 볼륨 데이터를 시각화하는 함수.

    Args:
        front_xray_tensor (torch.Tensor): 정면 X-ray 텐서 [1, 1, H, W].
        side_xray_tensor (torch.Tensor): 측면 X-ray 텐서 [1, 1, H, W].
        ct_volume_tensor (torch.Tensor): CT 볼륨 텐서 [1, 1, D, H, W].
    """
    # 텐서를 NumPy 배열로 변환
    front_xray = front_xray_tensor[0, 0].cpu().numpy()  # [H, W]
    side_xray = side_xray_tensor[0, 0].cpu().numpy()    # [H, W]
    ct_volume = ct_volume_tensor[0, 0].cpu().numpy()    # [D, H, W]

    # 정면 및 측면 X-ray 시각화
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(front_xray, cmap='gray')
    plt.title("Front X-ray")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(side_xray, cmap='gray')
    plt.title("Side X-ray")
    plt.axis('off')

    # CT 볼륨 중간 슬라이스 시각화
    mid_slice = ct_volume.shape[0] // 2  # 깊이의 중간 슬라이스 선택
    plt.subplot(1, 3, 3)
    plt.imshow(ct_volume[mid_slice, :, :], cmap='gray')
    plt.title(f"CT Volume (slice {mid_slice})")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # CT 볼륨 최대 강도 투영
    max_projection = np.max(ct_volume, axis=0)  # 깊이 방향으로 최대값 투영
    plt.figure(figsize=(6, 6))
    plt.imshow(max_projection, cmap='gray')
    plt.title("CT Volume Max Projection")
    plt.axis('off')
    plt.show()

# 시각화 함수 호출
visualize_data(front_xray_tensor, side_xray_tensor, ct_tensor)

